In [ ]:
# Calculating Accuracy using MAPE, RMSE, NSE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ipywidgets as widgets
from sklearn.metrics import mean_squared_error

# ==== PATHS ====
nequick2_path = "/content/drive/MyDrive/ISMR-SASTRA/EVENT_MONTHS_NEQUICK_DATA.xlsx"
real_path = "/content/drive/MyDrive/ISMR-SASTRA/EVENT MONTHS-REAL DATA.xlsx"
model_path = "/content/drive/MyDrive/ISMR-SASTRA/saits_vtec_model.pkl"

# ==== LOAD SAITS MODEL ====
with open(model_path, 'rb') as f:
    saits_model = pickle.load(f)

# ==== LOAD EXCEL FILES ====
nequick2_xls = pd.ExcelFile(nequick2_path)
real_xls = pd.ExcelFile(real_path)

# Months mapping
months = {
    "March": "MARCH",
    "June": "JUNE",
    "September": "SEPTEMBER",
    "December": "DECEMBER",
}

# ==== HELPER FUNCTIONS ====
def get_month_vtec(xls, sheet):
    df = pd.read_excel(xls, sheet_name=sheet, header=None)
    vtec_month = df.to_numpy(dtype=np.float32)
    if vtec_month.shape[0] < 24:
        vtec_month = np.pad(vtec_month, ((0,24-vtec_month.shape[0]), (0,0)), constant_values=np.nan)
    elif vtec_month.shape[0] > 24:
        vtec_month = vtec_month[:24, :]
    return vtec_month

def nse(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# ==== FUNCTION TO CALCULATE METRICS FOR BOTH MODELS SEPARATELY ====
def calculate_metrics_separately(month_label):
    sheet = months[month_label]

    # Load NEQUICK and Real data
    vtec_nequick2 = get_month_vtec(nequick2_xls, sheet)
    vtec_real = get_month_vtec(real_xls, sheet)

    # SAITS-impute missing values in Real
    vtec_saits_imputed = np.zeros_like(vtec_real)
    for day_idx in range(vtec_real.shape[1]):
        day_data = vtec_real[:, day_idx]
        mask = ~np.isnan(day_data)
        pred = saits_model.predict({"X": day_data.reshape(1, 24, 1)})
        imputed = pred["imputation"].flatten()
        vtec_saits_imputed[:, day_idx] = np.where(mask, day_data, imputed)

    # Monthly averages
    nequick2_avg = np.nanmean(vtec_nequick2, axis=1)
    saits_avg = np.nanmean(vtec_saits_imputed, axis=1)
    real_avg = np.nanmean(vtec_real, axis=1)

    # ---- Metrics for NEQUICK ----
    mape_nequick2 = mape(real_avg, nequick2_avg)
    rmse_nequick2 = np.sqrt(mean_squared_error(real_avg, nequick2_avg))
    nse_nequick2 = nse(real_avg, nequick2_avg) * 100

    # ---- Metrics for SAITS ----
    mape_saits = mape(real_avg, saits_avg)
    rmse_saits = np.sqrt(mean_squared_error(real_avg, saits_avg))
    nse_saits = nse(real_avg, saits_avg) * 100

    # Print metrics separately
    print(f"Metrics for {month_label}:")
    print(f"NEQUICK2 -> MAPE: {mape_nequick2:.3f} %, RMSE: {rmse_nequick2:.2f} TECU, NSE: {nse_nequick2:.3f} %")
    print(f"SAITS   -> MAPE: {mape_saits:.3f} %, RMSE: {rmse_saits:.2f} TECU, NSE: {nse_saits:.3f} %")

    # Plot for visual comparison
    plt.figure(figsize=(12,6))
    hours = range(1,25)
    plt.plot(hours, nequick2_avg, color='red', lw=2, label='NEQUICK2 Avg')
    plt.plot(hours, saits_avg, color='blue', lw=2, label='SAITS-Imputed Avg')
    plt.plot(hours, real_avg, color='green', lw=2, linestyle='--', label='Real Avg (Ground Truth)')
    plt.xlabel("Hour (1–24)")
    plt.ylabel("VTEC (TECU)")
    plt.title(f"Monthly-Averaged VTEC - {month_label}")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(hours)
    plt.ylim(0, max(np.nanmax(nequick2_avg), np.nanmax(saits_avg), np.nanmax(real_avg)) + 20)
    plt.tight_layout()
    plt.show()

# ==== Dropdown for Month Selection ====
month_dropdown = widgets.Dropdown(
    options=list(months.keys()),
    description='Select Month:',
    value='March'
)

widgets.interact(calculate_metrics_separately, month_label=month_dropdown)


